In [205]:
#install python-docx
!pip install python-docx

In [206]:
#import packages
from docx import Document
import pandas as pd
import numpy as np

In [207]:
#Getting students' info
info = pd.read_excel('Students_Info.xlsx')
info = pd.DataFrame(info)

#Clean info dataframe
info.rename(columns={'SEN\nCode':'SEN Code'}, inplace=True)
info['SEN Code'] = info['SEN Code'].astype(int)

#Set index
info.set_index(['SEN Code'], inplace=True)

info

,Class,Name,Difficulties,X1,X2
SEN Code,,,,,
56,5A,Student 12,Reading Disorder,VA,HMSC
57,5B,Student 33,Autism,VA,HMSC
58,5B,Student 37,Reading Disorder,Geog,NaN
59,5C,Student 50,"Autism, Language disorder",Chem,Bio
60,5C,Student 59,"Language disorder, Reading Disorder",NaN,THS
61,5C,Student 66,"Autism, Reading Disorder",NaN,Econ
62,5C,Student 63,Reading Disorder,ERS,NaN
63,5C,Student 67,Language disorder,ERS,NaN
64,5D,Student 68,Reading Disorder,NaN,Phy


In [208]:
#Read text in template file
doc = Document('template_s5.docx')
for para in doc.paragraphs:
    print(para.text)

ABC Secondary School
Students’ Support Measures
2021-2022 (1st term)

Name: 		Class: 
Difficulties:






In [209]:
#Read tables in template file
#Convert tables into one dataframe
table_list = []
for table in doc.tables:
    data = [[cell.text for cell in row.cells] for row in table.rows]
    table_list.append(pd.DataFrame(data))
df = pd.concat(table_list)

#Set header
df.columns = ['Item', 'Supporting Measure', 'Chi', 'Eng', 'Mat', 'LS', 'X1', 'X2', 'RE', 'PE', 'Avg', 'Position', 'Position']

#Clean the dataframe
df['Item'] = df['Item'].str.strip()
df['Supporting Measure'] = df['Supporting Measure'].str.strip()

#Set index
df.set_index('Supporting Measure', inplace=True)

df

,Item,Chi,Eng,Mat,LS,X1,X2,RE,PE,Avg,Position,Position
Supporting Measure,,,,,,,,,,,,
Supporting Measure,Item,Chi,Eng,Mat,LS,,,RE,PE,\t,\t,
Seat close to teacher,Classroom Arrangement,,,,,,,,,,,
Peer aids,Classroom Arrangement,,,,,,,,,,,
Quite place,Classroom Arrangement,,,,,,,,,,,
Seat close to blackboard,Classroom Arrangement,,,,,,,,,,,
Collaborative learning,Classroom Arrangement,,,,,,,,,,,
Differentiated colors for words,Pedagogical\nStrategy,,,,,,,,,,,
Short breaks,Pedagogical\nStrategy,,,,,,,,,,,
Informing test syllabus in advance,Pedagogical\nStrategy,,,,,,,,,,,


In [210]:
#Get items of supporting measures
items = []
for item in df['Item'].unique():
    if item not in ['Item', '']:
        item = item.replace('\n', ' ')
        items.append(item)
items

['Classroom Arrangement',
 'Pedagogical Strategy',
 'Multisensory Learning',
 'Assessment',
 'Composition',
 'Dictation']

In [211]:
#Read related columns in students' supports excel
supports = pd.read_excel('Supporting_Measures.xlsx')
supports = pd.DataFrame(supports)


#Clean students' supports dataframe
supports[['Record ID', 'SEN Code']] = supports[['Record ID', 'SEN Code']].astype(int)


#Clean subject names
for i in range(0,supports.shape[0]):
    subj = supports['Subject'][i]
    subj = subj.replace(';', '')
    subj = subj.replace(' ', '')
    if subj == 'Math':
        subj = 'Mat'
    supports.loc[i,'Subject'] = subj
    
supports['Subject'].unique()

array(['C.Hist', 'Science', 'Phy', 'Mandarin', 'Chi', 'IT', 'L&S', 'LS',
       'Bio', 'Mat', 'Eng', 'RE', 'Technology&Life', 'HMSC', 'THS',
       'Geog', 'Hist', 'Business', 'Chem', 'Econ', 'VA', 'ERS', 'PE'],
      dtype=object)

In [212]:
#Get unique elements of students' supporting measures
unique_supports = []
for item in items:
    for i in range(0, supports.shape[0]):
        temp = supports.loc[i, item].split(';')
        del temp[-1]
        for sup in temp:
            sup = sup.strip()
            if sup not in unique_supports:
                unique_supports.append(sup)

#Check whether the template includes all the supporting measures
check = 0
for sup in unique_supports:
    if sup not in df.index and sup != 'NA':
        print(sup + ' need to be added to the template.')
        check += 1
if check == 0:
    print('The measures in the template are complete.')

The measures in the template are complete.


In [213]:
#Read Exam Scores
S5marks_Exam = pd.read_excel('Exam_Scores.xlsx', sheet_name='S.5')
S5marks_Exam = pd.DataFrame(S5marks_Exam)

#Clean S5 Exam1 marks dataframe
S5marks_Exam[['No.', 'Pos']] = S5marks_Exam[['No.', 'Pos']].astype(int)

#Keep scores for Chi, Eng, Mat with full marks of 100
S5marks_Exam.drop(columns=['Chi', 'Eng', 'Math'], axis=1, inplace=True)
S5marks_Exam.rename(columns={'Chi(100)':'Chi', 'Eng(100)':'Eng', 'Math(100)':'Mat', 'Average':'Avg', 'Pos':'Position'}, inplace=True) #Template uses 'Mat' to stand for Math

S5marks_Exam

,Class,No.,Name,Total,Full,Avg,Position,Conduct,Chi,Eng,...,Chem,Bio,Bus,Econ,THS,ERS,VA,HMSC,RE,PE
0,5A,1,Student 1,490.0,900.0,54.44,29,A-,57.5,53.5,...,---,---,---,---,---,---,64.0,50.0,A,B
1,5A,2,Student 2,566.0,900.0,62.89,12,A-,58.5,56.5,...,---,---,72.0,---,63.0,---,---,---,A,C
2,5A,3,Student 3,561.0,900.0,62.33,13,A-,60.5,58.0,...,---,---,---,---,---,---,67.0,---,A,B
3,5A,4,Student 4,485.0,900.0,53.89,30,A-,58.0,49.5,...,---,---,---,---,---,---,66.0,50.0,A,C
4,5A,5,Student 5,533.0,900.0,59.22,17,A-,61.0,49.5,...,---,---,---,---,---,---,69.0,---,A,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,5D,18,Student 84,279.0,800.0,34.88,71,B,31.0,25.0,...,---,---,---,---,---,50.0,---,---,A,C
84,5D,19,Student 85,417.0,900.0,46.33,52,B+,49.0,14.0,...,---,---,---,---,50.0,---,---,---,A,C
85,5D,20,Student 86,301.0,900.0,33.44,74,B,35.0,17.5,...,---,---,---,---,---,53.0,---,---,B,B
86,5D,21,Student 87,314.0,900.0,34.89,70,B+,41.0,28.0,...,---,---,---,---,---,---,51.0,---,A,B


In [214]:
#Generate individual student supports form
for code in info.index: 
    sup = supports[supports['SEN Code'] == code].copy()
    sup = sup.reset_index(drop=True)
    df_ind = df.copy()
    rows = sup.shape[0]

#Write students' electives to columns
    if isinstance(info.loc[code, 'X1'], str) and isinstance(info.loc[code, 'X2'], str):
        X1 = info.loc[code, 'X1']
        X2 = info.loc[code, 'X2']
        df_ind.rename(columns={'X1':X1, 'X2':X2}, inplace=True)
        df_ind.loc['Supporting Measure',X1] = X1
        df_ind.loc['Supporting Measure',X2] = X2
    elif isinstance(info.loc[code, 'X1'], str):
        df_ind = df_ind[['Item', 'Chi', 'Eng', 'Mat', 'LS', 'X1', 'RE', 'PE', 'X2', 'Avg', 'Position']]
        X1 = info.loc[code, 'X1']
        df_ind.rename(columns={'X1':X1, 'X2':' '}, inplace=True)
        df_ind.loc['Supporting Measure',X1] = X1
    elif isinstance(info.loc[code, 'X2'], str):
        df_ind = df_ind[['Item', 'Chi', 'Eng', 'Mat', 'LS', 'X2', 'RE', 'PE', 'X1', 'Avg', 'Position']]
        X2 = info.loc[code, 'X2']
        df_ind.rename(columns={'X1':' ', 'X2':X2}, inplace=True)
        df_ind.loc['Supporting Measure',X2] = X2
    else:
        df_ind = df_ind[['Item', 'Chi', 'Eng', 'Mat', 'LS', 'RE', 'PE', 'X1', 'X2', 'Avg', 'Position']]
        df_ind.rename(columns={'X1':' ', 'X2':' '}, inplace=True)

#Put ticks in the boxes of corresponding measures adopted
    for item in items:
        for r in range(0,rows):
            item_sup = sup.loc[r,item].split(';')
            del item_sup[-1]
            for i in range(0, len(item_sup)):
                if item_sup[i] != 'NA' and sup['Subject'][r] in df_ind.columns:
                    item_sup[i] = item_sup[i].strip()
                    df_ind.loc[item_sup[i],sup['Subject'][r]] = 'v'

#Fill in marks
    name = info['Name'][code]
    gets = df_ind.columns
    mark_Exam = S5marks_Exam[S5marks_Exam['Name'] == name].copy()
    mark_Exam = mark_Exam.reset_index(drop=True)
    for get in gets:
        if get in mark_Exam.columns:
            df_ind.loc['1st term Exam', get] = str(mark_Exam.loc[0,get])
        
#Remove index
    df_ind.reset_index(inplace=True)
    df_ind.set_index('Item', inplace=True)
    df_ind.reset_index(inplace=True)
    df_ind.columns = range(0,df_ind.shape[1])
    
#Generate doc
    content = doc.paragraphs
    content[4].text = 'Name: ' + name + '\t\t' + 'Class: ' + info['Class'][code]
    content[5].text = 'Difficulties: ' + info['Difficulties'][code]
    for i in range(0, df_ind.shape[0]):
        for j in range(0, df_ind.shape[1]):
            table.cell(i,j).text = df_ind.iloc[i,j]
    filename = info['Class'][code] + ' ' + name
    doc.save(filename + '.docx')
    print(filename + ' done!')
print('S5 all done!')

5A Student 12 done!
5B Student 33 done!
5B Student 37 done!
5C Student 50 done!
5C Student 59 done!
5C Student 66 done!
5C Student 63 done!
5C Student 67 done!
5D Student 68 done!
5D Student 71 done!
S5 all done!
